In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import SimpleITK as sitk
import os

In [4]:
def get_attributes(sitk_image):
    """Get physical space attributes (meta-data) of the image."""
    attributes = {}
    attributes['orig_pixelid'] = sitk_image.GetPixelIDValue()
    attributes['orig_origin'] = sitk_image.GetOrigin()
    attributes['orig_direction'] = sitk_image.GetDirection()
    attributes['orig_spacing'] = np.array(sitk_image.GetSpacing())
    attributes['orig_size'] = np.array(sitk_image.GetSize(), dtype=int)
    return attributes


def resample_sitk_image(sitk_image,
                        new_spacing=[1, 1, 1],
                        new_size=None,
                        attributes=None,
                        interpolator=sitk.sitkLinear,
                        new_origin=(0.0, 0.0, 0.0),
                        fill_value=0):
    """
    Resample a SimpleITK Image.

    Parameters
    ----------
    sitk_image : sitk.Image
        An input image.
    new_spacing : list of int
        A distance between adjacent voxels in each dimension given in physical units (mm) for the output image.
    new_size : list of int or None
        A number of pixels per dimension of the output image. If None, `new_size` is computed based on the original
        input size, original spacing and new spacing.
    attributes : dict or None
        The desired output image's spatial domain (its meta-data). If None, the original image's meta-data is used.
    interpolator
        Available interpolators:
            - sitk.sitkNearestNeighbor : nearest
            - sitk.sitkLinear : linear
            - sitk.sitkGaussian : gaussian
            - sitk.sitkLabelGaussian : label_gaussian
            - sitk.sitkBSpline : bspline
            - sitk.sitkHammingWindowedSinc : hamming_sinc
            - sitk.sitkCosineWindowedSinc : cosine_windowed_sinc
            - sitk.sitkWelchWindowedSinc : welch_windowed_sinc
            - sitk.sitkLanczosWindowedSinc : lanczos_windowed_sinc
    fill_value : int or float
        A value used for padding, if the output image size is less than `new_size`.

    Returns
    -------
    sitk.Image
        The resampled image.

    Notes
    -----
    This implementation is based on https://github.com/deepmedic/SimpleITK-examples/blob/master/examples/resample_isotropically.py
    """
    sitk_interpolator = interpolator

    # provided attributes:
    if attributes:
        orig_pixelid = attributes['orig_pixelid']
        orig_origin = sitk_image.GetOrigin()
        orig_direction = attributes['orig_direction']
        orig_spacing = attributes['orig_spacing']
        orig_size = attributes['orig_size']

    else:
        # use original attributes:
        orig_pixelid = sitk_image.GetPixelIDValue()
        orig_origin = sitk_image.GetOrigin()
        orig_direction = sitk_image.GetDirection()
        orig_spacing = np.array(sitk_image.GetSpacing())
        orig_size = np.array(sitk_image.GetSize(), dtype=int)

    # new image size:
    if not new_size:
        new_size = orig_size * (orig_spacing / new_spacing)
        new_size = np.ceil(new_size).astype(int)  # Image dimensions are in integers
        new_size = [int(s) for s in new_size]  # SimpleITK expects lists, not ndarrays

    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetSize(new_size)
    resample_filter.SetTransform(sitk.Transform())
    resample_filter.SetInterpolator(sitk_interpolator)
    resample_filter.SetOutputOrigin(orig_origin)
    resample_filter.SetOutputSpacing(new_spacing)
    resample_filter.SetOutputDirection(orig_direction)
    resample_filter.SetDefaultPixelValue(fill_value)
    resample_filter.SetOutputPixelType(orig_pixelid)

    resampled_sitk_image = resample_filter.Execute(sitk_image)
    return resampled_sitk_image

In [2]:
path = '/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/'
directories = os.listdir(path)
print(directories)

['1-200', '201-400', '401-600', '601-800', '801-1000']


In [3]:
training_data = []

for d in directories:
    # print(d)
    if '1-200' in d:
        for i in range(200):
            i += 1
            img_path = path + d + '/' + str(i) + '.img.nii.gz'
            label_path = path + d + '/' + str(i) + '.label.nii.gz'
            id_ = d + '/' + str(i)
            data = {"id": id_, "image": img_path, "label": label_path}
            training_data.append(data)

    # elif '201-400' in d:
    #     for i in range(200):
    #         i += 201
    #         img_path = path + d + '/' + str(i) + '.img.nii.gz'
    #         label_path = path + d + '/' + str(i) + '.label.nii.gz'
    #         id_ = d + '/' + str(i)
    #         data = {"id": id_, "image": img_path, "label": label_path}
    #         training_data.append(data)
   
    # elif '401-600' in d:
    #     for i in range(200):
    #         i += 401
    #         img_path = path + d + '/' + str(i) + '.img.nii.gz'
    #         label_path = path + d + '/' + str(i) + '.label.nii.gz'
    #         id_ = d + '/' + str(i)
    #         data = {"id": id_, "image": img_path, "label": label_path}
    #         training_data.append(data)
    
    # elif '601-800' in d:
    #     for i in range(200):
    #         i += 601
    #         img_path = path + d + '/' + str(i) + '.img.nii.gz'
    #         label_path = path + d + '/' + str(i) + '.label.nii.gz'
    #         id_ = d + '/' + str(i)
    #         data = {"id": id_, "image": img_path, "label": label_path}
    #         training_data.append(data)
    
    # elif '801-1000' in d:
    #     for i in range(200):
    #         i += 801
    #         img_path = path + d + '/' + str(i) + '.img.nii.gz'
    #         label_path = path + d + '/' + str(i) + '.label.nii.gz'
    #         id_ = d + '/' + str(i)
    #         data = {"id": id_, "image": img_path, "label": label_path}
    #         training_data.append(data)
    

In [27]:
save_path = '/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/resampled_space/'
if not os.path.exists(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))

In [29]:
for p in training_data:
    print(p['image'])
    patient_id = p['id'].split('/')[-1]
    
    
    image = sitk.ReadImage(p['image'])
    label = sitk.ReadImage(p['label'])
    print(image.GetSize(), image.GetSpacing())

    # image_attributes = get_attributes(image)
    # label_attributes = get_attributes(label)

    new_spacing = [1, 1, 1]
    new_image = resample_sitk_image(image, new_spacing=new_spacing)
    new_label = resample_sitk_image(label, new_spacing=new_spacing, interpolator=sitk.sitkNearestNeighbor)
    print(new_image.GetSize(), new_image.GetSpacing())
    
    if not os.path.exists(os.path.join(save_path, patient_id)):
        os.makedirs(os.path.join(save_path, patient_id))

    sitk.WriteImage(new_image, os.path.join(save_path, patient_id, (patient_id+'_img.nii.gz')), True)
    sitk.WriteImage(new_label, os.path.join(save_path, patient_id, (patient_id+'_seg.nii.gz')), True)
    print('Resampled images are saved.')
    # break

/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/1-200/1.img.nii.gz
(512, 512, 275) (0.376953125, 0.376953125, 0.5)
(193, 193, 138) (1.0, 1.0, 1.0)
Resampled images are saved.
/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/1-200/2.img.nii.gz
(512, 512, 275) (0.37109375, 0.37109375, 0.5)
(190, 190, 138) (1.0, 1.0, 1.0)
Resampled images are saved.
/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/1-200/3.img.nii.gz
(512, 512, 275) (0.361328125, 0.361328125, 0.5)
(185, 185, 138) (1.0, 1.0, 1.0)
Resampled images are saved.
/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/1-200/4.img.nii.gz
(512, 512, 259) (0.341796875, 0.341796875, 0.5)
(175, 175, 130) (1.0, 1.0, 1.0)
Resampled images are saved.
/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/unzip_space/1-200/5.img.nii.gz
(512, 512, 206) (0.34765625, 0.3476